In [ ]:
!pip install bitsandbytes==0.43.1
!pip install accelerate==0.30.1
!pip install transformers==4.39.3
!pip install gradio==4.29.0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)

# Specify the model ID for the pre-trained model from Hugging Face
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load the tokenizer using the specified model ID
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Configure the BitsAndBytes settings for loading the model
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model for causal language modeling using the specified model ID
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # Use bfloat16 as the data type for the model
    device_map="auto",          # device_map="auto" automatically maps the model to available devices (like GPU if available)
    quantization_config=config  # quantization_config=config specifies the quantization configuration for the model
)

# Mutli Turn Prompt Engineering
- 여러 차례의 대화 턴(turn)을 통해 LLM에 정보를 제공하고,
이를 바탕으로 더욱 정교한 응답을 생성하도록 하는 Prompt Engineering 기법
- 자연스러운 대화 생성 가능 : Chatbot에서 대부분 지원되어야 함.
- 문맥 이해 강화 : 한 번의 턴 만을 고려하는 대신, 여러 턴의 대화를 고려하면 모델이 문맥을 더 잘 이해함
- 예시
    ```
    Input
    질문 : 미국 수도가 어디야??
    대답 : 미국의 수도는 워싱턴 D.C. 입니다.
    질문 : 그럼 한국은?

    Output
    대답 : 한국의 수도는 서울입니다.
    ```

In [ ]:
messages = [
    {"role": "system", "content": "You are a nice chatbot that helps users. You always have to respond briefly, within three sentences."},
    {"role": "user", "content": "What is the capital of the United States?"},
    {"role": "assistant", "content": "The capital of the United States is Washington D.C."},
    {"role": "user", "content": "Then, what about Korea?"}
]

# 토큰나이즈
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True, # 다음 추론을 자동으로 넣어는 옵션
    return_tensors="pt"
).to(model.device)


# Terminators 설정
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
# Output 
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

In [ ]:
# Input 토큰을 제외하고 응답 토큰 출력
response = outputs[0][input_ids.shape[-1]:]

# string으로 변환
print("repsonse : ", tokenizer.decode(response, skip_special_tokens=True))

# Zero-Shot Prompt Engineering
- LLM에 예제를 제공하지 않고 응답을 생성하게 하는 기법
- LLM 성능 평가 시, 보통 사용되는 기법

In [ ]:
# Define the initial conversation messages
messages = [
    {
        "role": "system",
        "content": "You are a Korea robot that summarizes documents. You MUST answer in Korea"
    },
    {
        "role": "user",
        "content": """
        ###document: 기후 변화는 수십 년에서 수백만 년에 걸친 기간 동안의 기상 패턴의 통계적 분포에서 장기적인 변화를 의미합니다.
        이는 평균 기상 조건의 변화, 또는 평균 조건 주변의 기상 분포의 변화를 의미할 수 있습니다.
        또한 이것은 기온, 강수량, 또는 바람 패턴의 변화를 포함할 수 있습니다.
        """
    }
]

# Remove newline characters from the content of each message
for message in messages:
    message["content"] = message["content"].replace("\n", "")

# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages,                    # List of conversation messages
    add_generation_prompt=True,  # Add a text generation prompt
    return_tensors="pt"          # Return the results as PyTorch tensors
).to(model.device)               # Move

# Define a list of terminators, which specifies the end-of-sequence token ID
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


In [ ]:
# Generate model outputs based on the input IDs
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

# Extract the generated response tokens, excluding the input prompt tokens
response = outputs[0][input_ids.shape[-1]:]

# Decode the response tokens back into a string, skipping special tokens
print("response : ", tokenizer.decode(response, skip_special_tokens=True))

# Few-Shot Prompt Engineering
- LLM에 몇 가지(Few) 예제를 제공하여 응답을 생성하게 하는 기법
- Zero-Shot 보다 더 나은 성능을 보임
- 예제 개수에 따라 , 1-shot, 3-shot 등으로 불림
- 일정 크기 이상의 LLM에 적용 가능
    ```
    ex)
    Input
    질문 : 너는 리뷰의 긍/부정을 알려주는 로봇이야 "오늘 시켰는데 정말 맛있었어요"
    대답 : "긍정"
    질문 : "별로 였어요"
    대답 : "부정"
    질문 : "한번 더 시켜먹고 싶어요"

    Output
    대답 : "긍정"
    ```

In [ ]:
# Define the initial conversation messages
messages = [
    {"role": "system", "content": "You are a korea robot that summarizes documents. You MUST answer in Korea"},
    {"role": "user", "content": """
    ###document: 에펠탑은 프랑스 파리의 샹 드 마르스에 위치한 철제 격자 탑입니다.
    이 탑은 그것을 설계하고 건설한 회사의 엔지니어인 구스타브 에펠타의 이름을 딴 것입니다.
    1887년부터 1889년까지 1889년 세계 박람회의 입구로 건설되었으며,
    처음에는 그 디자인을 이유로 프랑스의 주요 예술가들과 지식인들로부터 비판을 받았습니다.
    하지만 이제는 프랑스의 전세계적인 문화 아이콘 그리고 세계에서 가장 알아보기 쉬운 구조물 중 하나가 되었습니다.
    """},
    {"role": "assistant", "content": """에펠탑은 파리의 철제 격자 구조물로, 1887년부터 1889년까지 건설되었으며,
    이후로는 프랑스의 전 세계적으로 인식된 상징이 되었습니다."""},

    {"role": "user", "content": """
    ###document: 애플은 컴퓨터 소프트웨어와 온라인 서비스를 설계, 개발, 판매하는 미국의 다국적 기술 회사로,
     캘리포니아 주 쿠퍼티노에 본사를 두고 있습니다.
     애플은 아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크 기술 회사로 간주됩니다.
    """},
    {"role": "assistant", "content": """애플은 소비자 전자 제품과 소프트웨어를 설계하고 판매하는 주요 기술 회사로,
    아마존, 구글, 마이크로소프트, 페이스북과 함께 빅 테크로 간주됩니다."""},

    {"role": "user", "content": """
    ###document: 기후 변화는 수십 년에서 수백만 년에 걸친 기간 동안의 기상 패턴의 통계적 분포에서 장기적인 변화를 의미합니다.
    이는 평균 기상 조건의 변화, 또는 평균 조건 주변의 기상 분포의 변화를 의미할 수 있습니다.
    또한 이것은 기온, 강수량, 또는 바람 패턴의 변화를 포함할 수 있습니다.
    """}
]

# Remove newline characters from the content of each message
for message in messages:
    message["content"] = message["content"].replace("\n", "")


# Tokenize the prompt
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)



In [ ]:
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)


In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))